In [3]:
import time
import random
import numpy as np

import altair as alt
import pandas as pd
import collections
from causality_functions import jitter, dependent, simulations_data
from causality_functions import compute_regression, compute_correlation
from causality_functions import add_confidence_stats, confidence_graphs
from causality_functions import stats_graphs

alt.data_transformers.disable_max_rows()

samplesize = 500
Am, Bm, Cm, = 40, 10, 30
Xm, Ym = 1, 1
Astd,Bstd,Cstd = 7, 5, 11
Xstd,Ystd = 1, 1

m1, k1 = 1, -1.0
m2, k2 = 1.5, 2.0
m3, k3 = 1, -1.5
m4, k4 = .75, 0.0

d1 = 1
d2 = 1
d3 = 1
d4 = 1

errorA = 4
errorB = 8
errorC = 2
errorX = 0.75
errorY = 0.6

mrange1 = 0,5 #slope ranges
mrange2 = 0,5
mrange3 = 0,5
mrange4 = 0,5

erangeA = 0.2,2 #error range
erangeB = 0.2,2 #error range
erangeC = 0.2,10 #error range
erangeX = 0.2,2 #error range
erangeY = 0.2,2 #error range


def linear_positive_C_to_B(n=100):

    def next_generation(A):
        A_ = jitter(A, errorA) # previous A + error
        B_ = dependent(A_, m1, k1, error=errorB)
        C_ = dependent(B_, m2, k2, error=errorC)
        return A_, B_, C_

    def next_generation_delayed(history):
        A, _, _ = history[-1]
        A_ = jitter(A, errorA) # previous A + error
        A, _, _ = history[1 - d1]
        _, _, C = history[1 - d3]
        B_ = jitter(A*m1 + m3*C + k1 ,  errorB)
        _, B, _ = history[1 - d2]
        C_ = dependent(B, m2, k2, error=errorC)
        return A_, B_, C_

    A = np.random.normal(loc=Am, scale=Astd) # normal distribution, Am - mean and Astd - standard deviation
    history = collections.deque(maxlen=max([d1, d2, d3]))
    for i in range(max([d1, d2, d3])):
        history.append(next_generation(A))

    for i in range(n):
        history.append(next_generation_delayed(history))

    return np.array(history[-1])


def linear_positive_A_to_C(n=100):

    def next_generation(A):
        A_ = jitter(A, errorA) # previous A + error
        B_ = dependent(A_, m1, k1, error=errorB)
        C_ = dependent(B_, m2, k2, error=errorC)
        return A_, B_, C_

    def next_generation_delayed(history):
        A, _, _ = history[-1]
        A_ = jitter(A, errorA) # previous A + error
        A, _, _ = history[1 - d1]
        B_ = dependent(A, m1, k1 ,  errorB)
        _, B, _ = history[1 - d2]
        A, _, _ = history[1 - d3]
        C_ = jitter(A*m3+ B*m2+ k2, errorC)
        return A_, B_, C_

    A = np.random.normal(loc=Am, scale=Astd) # normal distribution, Am - mean and Astd - standard deviation # 0-1 , uniformly distributed
    history = collections.deque(maxlen=max([d1, d2, d3]))
    for i in range(max([d1, d2, d3])):
        history.append(next_generation(A))

    for i in range(n):
        history.append(next_generation_delayed(history))

    return np.array(history[-1])


def update_slopes_errors():
    def select(lower, upper):
        return lower + random.random()*(upper-lower)
    
    global m1, m2, m3, m4
    global errorA, errorB, errorC, errorX, errorY

    m1 = select(*mrange1)
    m2 = select(*mrange2)
    m3 = select(*mrange3)
    m4 = select(*mrange4)
    
    errorA = select(*erangeA)
    errorB = select(*erangeB)
    errorC = select(*erangeC)
    errorX = select(*erangeX)
    errorY = select(*erangeY)

def overall_simulation(n=100):
    stats = []
    data = []
    for i in range(n):
        update_slopes_errors()
        ABC = simulations_data(linear_positive_C_to_B, samplesize) # you can change pathway here
        r = compute_regression(ABC)
        r.update(compute_correlation(ABC))
        stats.append(r)
        data.append(ABC)
    return pd.DataFrame(stats), np.array(data)

d, ABC_all = overall_simulation()

In [4]:
add_confidence_stats(d, ABC_all)
confidence_graphs(d)

alt.VConcatChart(...)

In [5]:
stats_graphs(d)

alt.VConcatChart(...)

In [6]:
A, B, C = random.choice(ABC_all).transpose()
AB = alt.Chart(pd.DataFrame({"A":A, "B":B})).mark_circle().encode(
    x="A",
    y="B")
BC= alt.Chart(pd.DataFrame({"B":B, "C":C})).mark_circle().encode(
    x="B",
    y="C")
AC = alt.Chart(pd.DataFrame({"A":A, "C":C})).mark_circle().encode(
    x="A",
    y="C")

alt.vconcat(AB, BC, AC)

alt.VConcatChart(...)

In [7]:
d

,kAB,kBC,kAC,mAB,mBC,mAC,r_sqrAB,r_sqrBC,r_sqrAC,r_E,...,rAB,rBC,rAC,rAB2*rBC2-rAC2,r_E_BA_C2-rBC2,mAB*mBC-mAC,confidence,confidence_residual_corr,confidence_corrected_bc_corr,confidence_slope_AC
0,-1.078234e+00,3.017377e+00,6.148251e-01,1.320047e-01,0.090012,7.021437e-02,0.028908,0.003613,0.003647,-0.008672,...,0.170022,0.060110,0.060393,-0.003543,-0.002592,-5.833240e-02,within,within,within,within
1,8.965843e+29,4.063284e+28,7.301583e+29,3.814935e+27,0.788566,3.444890e+27,0.132941,0.855099,0.149065,-0.050330,...,0.364610,0.924716,0.386089,-0.035387,-0.851969,-4.365609e+26,more,within,more,within
2,2.720727e+54,5.249737e+53,6.856410e+54,3.526897e+52,2.372026,8.674194e+52,0.312716,0.917186,0.308348,-0.046262,...,0.559210,0.957698,0.555291,-0.021529,-0.917140,-3.083026e+51,more,within,more,within
3,8.823069e-02,2.430321e+00,2.971522e+00,4.464594e-01,0.020508,-4.033198e-03,0.510910,0.000620,0.000061,0.026216,...,0.714780,0.024904,-0.007841,0.000255,0.000137,1.318906e-02,within,within,within,within
4,5.335007e+22,-2.773124e+20,2.438706e+23,1.656115e+21,4.566511,7.549485e+21,0.554719,0.996696,0.550960,0.025169,...,0.744795,0.998347,0.742267,0.001926,-0.993125,1.318318e+19,more,within,more,within
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,8.252502e+35,3.762866e+35,1.214450e+36,2.677078e+33,1.089744,4.428779e+33,0.081049,0.495751,0.092600,-0.043433,...,0.284692,0.704096,0.304302,-0.052420,-0.493055,-1.511450e+33,more,within,more,within
96,2.050369e+14,2.115454e+13,8.962666e+13,7.923768e+11,0.356270,3.999876e+11,0.122757,0.588334,0.144991,-0.065320,...,0.350367,0.767029,0.380777,-0.072769,-0.584936,-1.176875e+11,more,within,more,within
97,2.518352e+05,1.196401e+04,5.656164e+05,3.317499e+03,2.189851,7.209968e+03,0.425688,0.997791,0.418357,0.090147,...,0.652448,0.998895,0.646805,0.006391,-0.995597,5.485964e+01,more,more,more,within
98,1.472172e+00,6.110864e+00,6.527303e+00,3.991394e+00,0.511658,2.050794e+00,0.985012,0.906457,0.900376,-0.105029,...,0.992478,0.952080,0.948881,-0.007504,-0.906296,-8.563914e-03,more,less,more,within
